# Ch10 機器視覺實戰演練：CNN (Convolutional Neural Network)

#### 10.3.1 匯入模組

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D # new!

#### 10.3.2 載入資料集並做資料預處理

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
X_train = X_train.reshape(60000, 28, 28, 1).astype('float32')
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32')

In [5]:
X_train /= 255
X_test /= 255

In [6]:
n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

#### 10.3.3 規劃 CNN 模型的架構

In [7]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

#### 10.3.4 編譯、訓練模型

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
#註：由於神經網路的初始權重參數是隨機設定的, 參雜了隨機性, 因此底下 (或您重跑一次) 的結果不會與書中完全一樣, 但模型的能力是相近的
model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(X_test, y_test))

Epoch 1/10
469/469 [==============================] - 123s 260ms/step - loss: 0.2367 - accuracy: 0.9278 - val_loss: 0.0500 - val_accuracy: 0.9834
Epoch 2/10
469/469 [==============================] - 121s 258ms/step - loss: 0.0845 - accuracy: 0.9753 - val_loss: 0.0384 - val_accuracy: 0.9875
Epoch 3/10
469/469 [==============================] - 121s 258ms/step - loss: 0.0630 - accuracy: 0.9811 - val_loss: 0.0342 - val_accuracy: 0.9886
Epoch 4/10
469/469 [==============================] - 121s 257ms/step - loss: 0.0524 - accuracy: 0.9840 - val_loss: 0.0344 - val_accuracy: 0.9888
Epoch 5/10
469/469 [==============================] - 121s 258ms/step - loss: 0.0437 - accuracy: 0.9865 - val_loss: 0.0294 - val_accuracy: 0.9908
Epoch 6/10
469/469 [==============================] - 121s 259ms/step - loss: 0.0385 - accuracy: 0.9880 - val_loss: 0.0293 - val_accuracy: 0.9905
Epoch 7/10
469/469 [==============================] - 121s 259ms/step - loss: 0.0352 - accuracy: 0.9885 - val_loss: 0.0273 -